In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

## Read Data

In [3]:
train_images = np.load('NF_MNIST_train.npy')
train_labels = np.load('NF_MNIST_labels.npy')
test_images = np.load('NF_MNIST_test.npy')

## Data Pre-Processing

In [4]:
# Reshape the image data to include the channel dimension
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

# Normalize the image data
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, 10)

## Create Model

In [6]:
model = Sequential()
# Add convolution 2D
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))


model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
train_model = model.fit(train_images, train_labels,
                        batch_size=128,
                        epochs=100,
                        verbose=1,
                        validation_split=0.2,  # Use 20% of the data for validation
                        callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ])

## Dump Answer

In [8]:
# Process predictions
predictions = model.predict(test_images)

predicted_classes = np.argmax(predictions, axis=1)

313/313 [==============================] - 1s 1ms/step


In [9]:
np.savetxt('submission1.csv', predicted_classes, fmt='%d', delimiter=',')